In [29]:
from numpy import polyval
import numpy as np
import pandas as pd
import numpy.linalg as sp

In [ ]:
def secondordfunc(MEAout_xt):
    MEAout_xt2o = np.zeros((MEAout_xt.shape[0], 0), dtype=float)
    for ii2 in np.arange(MEAout_xt.shape[1] - 1):
        # idxtemp=np.arange(MEAout_xt.shape[1]-1)+ii2*(MEAout_xt.shape[1]-1)
        MEAtempmat = MEAout_xt[:, 1:] * MEAout_xt[:, (ii2 + 1):(ii2 + 2)]
        MEAout_xt2o = np.concatenate((MEAout_xt2o, MEAtempmat[:, 0:(ii2 + 1)]), axis=1)
    return MEAout_xt2o


def poly_surr(modout_xt, modout_ysim, beta_thresh=0.04):
    # standardize between 0-1
    xt_min = modout_xt.min(axis=0)
    xt_ptp = modout_xt.ptp(axis=0)
    modout_xt = (modout_xt - xt_min) / xt_ptp
    modout_xt = np.concatenate((0.5 * np.ones((modout_xt.shape[0], 1), dtype=float), modout_xt), axis=1)
    modout_ysim = modout_ysim / 100

    # first order surrogate
    b = np.dot(modout_xt.T, modout_xt)
    np.savetxt('crap.txt', b)
    
    b = np.loadtxt('crap.txt')
    
    beta = np.dot(np.linalg.inv(b), np.dot(modout_xt.T, modout_ysim))
    modout_pred = modout_xt @ beta

    # second order surrogate
    idx1 = np.squeeze(np.abs(beta) > beta_thresh)
    modout_xtsm = modout_xt[:, np.squeeze(np.abs(beta) > 4e-2)]
    modout_xt2o = secondordfunc(modout_xtsm)
    modout_xt2 = np.concatenate((modout_xt, modout_xt2o), axis=1)
    
    c = np.dot(modout_xt2.T, modout_xt2)
    
    np.savetxt('crap2.txt', c)
    
    c = np.loadtxt('crap2.txt')
    
    beta2 = sp.inv(c) @ (modout_xt2.T @ modout_ysim)
    return modout_xt2, beta2


des1 = np.array(pd.read_csv('Run_Results.csv', index_col=0))

modout_xt = des1[:, 0:5]
modout_ysim = des1[:, -2]

modout_xt2, beta2 = poly_surr(modout_xt, modout_ysim)

pred2 = 100 * modout_xt2 @ beta2

print(pred2)

In [ ]:
from sklearn.linear_model import LinearRegression

X = des1[:, 0:5]
y = des1[:, -2]


reg = LinearRegression()

reg.fit(X, y)

a, b, c, d, e = reg.coef_

def surr(X):
    return a*X[4] + b*X[3] + c*X[2] + d*X[1] + e*X[0]

print(surr(X[0]))

In [49]:
from sklearn.linear_model import LinearRegression

X = des1[:, 0:5]
y = des1[:, -2]


reg = LinearRegression()

reg.fit(X, y)

a, b, c, d, e = reg.coef_

def surr(X):
    return a*X[4] + b*X[3] + c*X[2] + d*X[1] + e*X[0]

print(surr(X[0]))

[93.99 88.0 89.55 90.43 94.83 80.49 62.07 83.64 77.5 93.51 93.67 94.73
 84.7 86.82 95.17 94.39 92.37]
12.70656378429257
